# The solow model and the solow model with human capital 

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **model analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `modelproject.py` file includes a function which could be used multiple times in this notebook.

Imports and set magics:

In [1]:
import numpy as np
import sympy as sm
from scipy import optimize
from sympy import Symbol
from sympy.solvers import solve
from IPython.display import display
import matplotlib.pyplot as plt
import ipywidgets as widgets

# output
sm.init_printing(use_unicode=True)

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# output
sm.init_printing(use_unicode=True)

# local modules
import modelproject

# The basic solow Model 

**Write out the model in equations here.** 

In the basic solow model we have:

$K_t$ as capital\
$L_t$ as labor\


The model also contains\
The GDP\
$Y_t =F(K_t,L_t)$\
$Y_t= B*K_t^\alpha*L_1^{1-\alpha}$\
$r_t=\alpha*B*\left(\frac{K_t}{K_t}\right)$\
$w_t=(1-\alpha) B\left(\frac{K_t}{L_t}\right)^{\alpha}$\
The savings\
$S_t=sY_t,s \in (0,1)$\
$K_{t+1}-K_t=S_t-\delta K_t$\
where we have: $k_t=S_t+(1-\delta)K_t$\
$L_{t+1}=(1+n)L_t$

In the basic solow model we can use this notation for per capita.\

$y_t=\left(\frac{Y_t}{L_t}\right)$ 
$k_t=\left(frac{K_t}{L_t}\right)$
    

## Analytical solution

If your model allows for an analytical solution, you should provide here.

You may use Sympy for this. Then you can characterize the solution as a function of a parameter of the model.

To characterize the solution, first derive a steady state equation as a function of a parameter using Sympy.solve and then turn it into a python function by Sympy.lambdify. See the lecture notes for details. 

In [ ]:
#define parameter
n,s,B,kstar,ystar,alpha,delta=sm.symbols('n s B k k^* y^* alpha delta')

#Here we will solve the equation for k
Solow_eq = sm.Eq(((s*B*k**alpha-(n + delta)*k)/(1 + n)),0)

#Now it is possible to find the steady-state for k 
k_s=sm.solve(Solow_eq,k)[0]
s_s_k=sm.Eq(kstar,k_s)

# Define y 
y=B*k**alpha

# Find the steady state for y
y_s=y.subs(k,k_s)
s_s_y=sm.Eq(ystar,y_s)

#

## Numerical solution

You can always solve a model numerically. 

Define first the set of parameters you need. 

Then choose one of the optimization algorithms that we have gone through in the lectures based on what you think is most fitting for your model.

Are there any problems with convergence? Does the model converge for all starting values? Make a lot of testing to figure these things out. 

# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

# Conclusion

Add concise conclusion. 